Note: This notebook uses 'btap' environment 
# Data Structures  

- [Lists](#lists)
- [Tuple](#tuples)
- [Dictionaries](#dictionaries)
- [ChainMaps](#chainmaps)
- [Sets](#sets)

In [113]:
def header_print(header, print_item, max_length=30):
    if len(header) >= max_length: 
        raise ValueError('Length of header too long')
    string = "{header" + ':' + str(max_length) + "}" + ": "
    print(string.format(header=header) + str(print_item))

### _Lists_

<div style='text-align: right; font-size: small;'>
<a href="#data-structures">to top</a>
</div>

In [1]:
some_list = ['10', 2, 'string', ['another', 'list']]

In [2]:
copied_list = some_list.copy()

In [3]:
copied_list[0] = 2
copied_list

[2, 2, 'string', ['another', 'list']]

In [4]:
some_list

['10', 2, 'string', ['another', 'list']]

In [5]:
copied_list[3][1] = 'screw up' 

In [6]:
some_list

['10', 2, 'string', ['another', 'screw up']]

In [7]:
copied_list

[2, 2, 'string', ['another', 'screw up']]

Findings: 
>A shallow copy means constructing a new collection object and then populating it with references to the child objects found in the original. In essence, a shallow copy is only one level deep. The copying process does not recurse and therefore won’t create copies of the child objects themselves. [Definition from realpython](https://realpython.com/copying-python-objects/)  

>A shallow copy constructs a new compound object and then (to the extent possible) inserts references into it to the objects found in the original. [Definition from python documentation](https://docs.python.org/3/library/copy.html)  
  
There is a list in `some_list` but that nested list doesn't exist within `some_list` itself, instead a reference to some place in memory where the nested list exists is stored within `some_list`. When a shallow copy is performed, new references to new objects belonging to primitive data types are created but references to child objects are retained. Therefore, `copied_list` would contain a reference to the **same nested list** and any changes to that nested list via `copied_list` would also be reflected in `some_list`.

In [8]:
from copy import deepcopy

another_list = ['10', 2, 'string', ['nested', 'list']]
deep_copied_list = deepcopy(another_list)

In [9]:
print(another_list)
print(deep_copied_list)

['10', 2, 'string', ['nested', 'list']]
['10', 2, 'string', ['nested', 'list']]


In [11]:
deep_copied_list[0] = 2
deep_copied_list

[2, 2, 'string', ['nested', 'list']]

In [12]:
deep_copied_list[3][1] = 'screw up'
deep_copied_list

[2, 2, 'string', ['nested', 'screw up']]

In [13]:
another_list

['10', 2, 'string', ['nested', 'list']]

Findings:  
> A deep copy makes the copying process recursive. It means first constructing a new collection object and then recursively populating it with copies of the child objects found in the original. Copying an object this way walks the whole object tree to create a fully independent clone of the original object and all of its children. [Definition from RealPython](https://realpython.com/copying-python-objects/)  

> A deep copy constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original. [Definition from python documentation](https://docs.python.org/3/library/copy.html)

A deep copy goes down the layers of nested child objects and creates a copy for each of those such that a new object in memory is created for each child object that the copied list then points to. 

In [50]:
recursive_list = ['element1', 'element2']
recursive_list.append(recursive_list)

In [51]:
deep_copy_rec_list = deepcopy(recursive_list)
shallow_copy_rec_list = recursive_list.copy()

In [52]:
deep_copy_rec_list

['element1', 'element2', [...]]

In [53]:
import sys 
print('deep copy', sys.getsizeof(deep_copy_rec_list))
print('shallow copy', sys.getsizeof(shallow_copy_rec_list))
print('original', sys.getsizeof(recursive_list))

deep copy 88
shallow copy 80
original 120


In [54]:
deep_copy_rec_list[2][2][2][2][2][2][2][2][1] = 'deep_virus'

In [55]:
deep_copy_rec_list

['element1', 'deep_virus', [...]]

In [56]:
recursive_list

['element1', 'element2', [...]]

In [57]:
shallow_copy_rec_list[2][2][2][2][2][2][2][2][1] = 'shallow_virus'

In [58]:
shallow_copy_rec_list

['element1', 'element2', ['element1', 'shallow_virus', [...]]]

In [59]:
recursive_list

['element1', 'shallow_virus', [...]]

Findings:  

Here we get to observe the difference in behaviour of a deep copy and a shallow copy, as well as how python stores items.  

`recursive_list` is created with 2 string elements followed by **a reference to itself**.  
`deep_copy_rec_list` is a deep copy of `recursive_list`  
`shallow_copy_rec_list` is a shallow copy of `recursive_list`  

So we get to see first how a copy works on references and also how editing an element in a mutable data structure works.  

You may think of `recursive_list` as something that looks like this:  

ObjectHeader - 'element1' - 'element2' - Ref(ObjectHeader)  
  
If we try to access Ref(ObjectHeader) by doing something like `recursive_list[2]`, we just get the exact same item.  

Now, if we do a deep copy, a whole new object is created "recursively" like so:  

DeepObjectHeader - 'element1' - 'element2' - Ref(DeepObjectHeader)

When we do `deep_copy_rec_list[2][2][2][2][2][2][2][2][1] = 'deep_virus'` we are actually just accessing the same object over and over again, eventually editing 'element2' and changing it to 'deep_virus'.  

When we do that, the header `DeepObjectHeader` is preserved with 'element2' being mutated to 'deep_virus'. As such we get:  

DeepObjectHeader - 'element1' - 'deep_virus' - Ref(DeepObjectHeader)  
  

Now, moving on to the shallow copy. Recalling the above definition, child object references are preserved (or, only the first level is copied) so what we get is:  

ShallowObjectHeader - 'element1' - 'element2' - Ref(ObjectHeader)  

When we do `shallow_copy_rec_list[2][2][2][2][2][2][2][2][1] = 'shallow_virus'`, we are not accessing `ShallowObjectHeader` but `ObjectHeader` which is the original object. Editing that causes `ObjectHeader` to change from:  

ObjectHeader - 'element1' - 'element2' - Ref(ObjectHeader)  
**_TO_**  
ObjectHeader - 'element1' - 'shallow_virus' - Ref(ObjectHeader)  

while `ShallowObjectHeader` remains as :  

ShallowObjectHeader - 'element1' - 'element2' - Ref(ObjectHeader)  

This is what gives us:  
`['element1', 'element2', ['element1', 'shallow_virus', [...]]]`  
for the shallow copy.  
And `['element1', 'shallow_virus', [...]]` for the original recursive list.

In [99]:
# Proof:
print(id(shallow_copy_rec_list))
print(id(recursive_list))
print(id(shallow_copy_rec_list[2]))
print(id(deep_copy_rec_list))
print(id(deep_copy_rec_list[2]))

2916041219264
2916041035136
2916041035136
2916041565568
2916041565568


### _Tuples_

<div style='text-align: right; font-size: small;'>
<a href="#data-structures">to top</a>
</div>

In [79]:
def hashable(object):
    try: 
        hash(object)
    except TypeError as e:
        return False 
    return True 

In [82]:
some_tuple = (10, 'alpha', ['mutable', 'list', 'with', 'hashable', 'elements'])
# do note that a list is actual non-hashable 
hashable(['some', 'list'])

False

In [83]:
hashable(some_tuple)

False

In [86]:
good_tuple = (10, 'alpha', 'plus', 'ultra')
hashable(good_tuple)

True

Findings:  
An object is hashable if its value cannot ever change. A hashable object can be used as a key in a dictionary etc., because it can guarantee that when it is referred to, it will always point to the same object in memory.  

Note that for `some_tuple`, although we say that tuples are 'immutable' (and it is), it is still unhashable, i.e., its value is _unfixed_. This is because of the mutable list that exists as one of its members. 

In [87]:
some_tuple[2][1] = 'virus'

In [89]:
some_tuple # Note that we can change mutable elements of an "immutable" tuple 

(10, 'alpha', ['mutable', 'virus', 'with', 'hashable', 'elements'])

In [90]:
some_dict = {good_tuple: True, some_tuple: False}

TypeError: unhashable type: 'list'

In [91]:
some_dict = {good_tuple: True}

In [92]:
some_dict[(10, 'alpha', 'plus', 'ultra')]

True

Findings:  
Note from the above examples that the non-hashable tuple `some_tuple` cannot be used as a dictionary key but the hashable tuple, `good_tuple` (comprising of primitive data types) can.  
Furthermore, we can grab the corresponding value for `good_tuple` in `some_dict` by grabbing it using another tuple with the exact same elements. This is because these 2 objects would have the same hash and would therefore point to the same location in the hash table. 

In [101]:
t = (1, 2, [30, 40])
t[2] + [50, 60]

[30, 40, 50, 60]

In [102]:
t

(1, 2, [30, 40])

In [103]:
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [115]:
t

(1, 2, [30, 40, 50, 60])

Findings:  
Notice that the in-place assignment acutally takes place even though an error was thrown. This tells us that the above operation is not atomic. In one step, a portion of it succeeds, and a portion of it fails, and the successful portion persists even though the overall operation has failed.

In [113]:
import dis
s = (1, 2, [30, 40])
x = [50, 60]

In [114]:
dis.dis('s[2] += x')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_CONST               0 (2)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                1 (x)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               1 (None)
             18 RETURN_VALUE


From the above we see that the INPLACE_ADD takes place before STORE_SUBSCR. And it is actually STORE_SUBSCR that fails for the tuple.

In [116]:
test_tuple = (1, 2, [30, 40])
print(id(test_tuple[2]))

2916063338688


In [117]:
test_tuple[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [118]:
test_tuple

(1, 2, [30, 40, 50, 60])

In [119]:
print(id(test_tuple[2]))

2916063338688


In [107]:
a = [30, 40]
b = [50, 60]
print(id(a))
print(id(b))

2916063290496
2916063338240


In [108]:
a += b
print(a)
print(id(a))

[30, 40, 50, 60]
2916063290496


In [109]:
a = a + b 
print(id(a))

2916063163904


Findings:  
An in-place operation changes the object while retaining it's header in memory.  

Re-assignments of a variable after an operation will cause a new object to be generated instead. 


### _Dictionaries_

<div style='text-align: right; font-size: small;'>
<a href="#data-structures">to top</a>
</div>

#### Unpacking

In [87]:


def dump(**kwargs):
    return kwargs 

d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4, 'e':5}
d3 = {'a': 6, 'd': 2}

header_print('Multiple dict unpackings', dump(**d1, **d2))
header_print('When keys overlap', dump(**d2, **d2, **d3))

Multiple dict unpackings      : {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}


TypeError: __main__.dump() got multiple values for keyword argument 'c'

In [15]:
header_print('Keys overlap (dict literal)', {**d1, **d2, **d3})

Keys overlap (dict literal)   : {'a': 6, 'b': 2, 'c': 3, 'd': 2, 'e': 5}


In [19]:
header_print('Merging maps', d1 | d3)
header_print('Merging goes left-to-right', d1 | d3 | d1)

Merging maps                  : {'a': 6, 'b': 2, 'd': 2}
Merging goes left-to-right    : {'a': 1, 'b': 2, 'd': 2}


In [24]:
header_print('d1 ID before merge', id(d1))
header_print('ID of merged item', id(d1 | d3))
print('perform in-place merge of d1')
d1 |= d3 
header_print('d1 contents after in-place', d1)
header_print('di ID after merge', id(d1))

d1 ID before merge            : 2648110609216
ID of merged item             : 2648109588864
perform in-place merge of d1
d1 contents after in-place    : {'a': 6, 'b': 2, 'd': 2}
di ID after merge             : 2648110609216


In [43]:
from collections import OrderedDict

d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4, 'e':5}
d3 = {'a': 6, 'd': 2}
d4 = OrderedDict()
d4 = {'f': 7, 'g': 8, 'h': 9}

list_of_dicts = [d1, d2, d3, d4]

for record in list_of_dicts: 
    match record: 
        case {'a': value_a, 'd': value_d}:
            print('matches case1: ', record)
        case {'b': value_b}: # partial_matching works
            print('matches case2: ', record)
        case {'e': value_e, 'c': value_c}: # order of keys matching does not matter
            print('matches case3: ', record)
        case {'h': value_h, 'f': value_f}: # and it does not matter even if the record is an instance of OrderedDicta
            print('matches case4: ', record) 
        case _:
            print('does not match', record)


matches case2:  {'a': 1, 'b': 2}
matches case3:  {'c': 3, 'd': 4, 'e': 5}
matches case1:  {'a': 6, 'd': 2}
matches case4:  {'f': 7, 'g': 8, 'h': 9}


In [45]:
from collections import abc 
my_dict = {}
isinstance(my_dict, abc.Mapping)
isinstance(my_dict, abc.MutableMapping)

True

In [48]:
from collections import UserDict 

class UpperCaseDict(UserDict):
    def __setitem__(self, key, value):
        key = key.upper() 
        super().__setitem__(key, value)

In [49]:
upper_case_dict = UpperCaseDict({'down': 1, 'with': 2, 'the': 3, 'sickness': 4})
upper_case_dict

{'DOWN': 1, 'WITH': 2, 'THE': 3, 'SICKNESS': 4}

In [50]:
print(isinstance(upper_case_dict, abc.Mapping))
print(isinstance(upper_case_dict, abc.MutableMapping))

True
True


The idea:  

One should endeavor to master the features that ABCs have. When creating custom objects that should behave like those ABCs, doing isinstance on these ABCs is good practice because it guarantees users that your custom object will have certain features

#### Experimentation

##### Hypothesis: Implementing all the components of the sequence protocol would cause isinstance(custom_object, abc.Sequence) to return True

In [68]:
class SampleSequence: 
    def __init__(self, initialdata=None):
        if initialdata is not None and not isinstance(initialdata, abc.Sequence):
            raise TypeError(f"Was expecting initialdata of type Sequence, received {type(initialdata)} instead")
        self.data = initialdata 

    def __repr__(self):
        return repr(self.data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __contains__(self, value):
        return value in self.data 
    
    def __iter__(self):
        return iter(self.data)
    
    def __reversed__(self):
        return self.data.reverse()
    
    def count(self):
        return len(self.data)
    
    def index(self, value):
        return self.data.index(value)


In [69]:
sample_sequence = SampleSequence(['a', 'couple', 'of', 'values'])
print(isinstance(sample_sequence, abc.Sequence))

False


##### Result: This is not true, after further research, it seems that we need to register our object into the virtual subcalss that is abc.Sequence for isinstance(custom_object, abc.Sequence) to return True  

The above will be investigated further in a later chapter

In [71]:
from collections import UserList
class AnotherSampleSequence(UserList): 
    def __init__(self, initialdata=None):
        if initialdata is not None and not isinstance(initialdata, abc.Sequence):
            raise TypeError(f"Was expecting initialdata of type Sequence, received {type(initialdata)} instead")
        self.data = initialdata 

    def __repr__(self):
        return repr(self.data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __contains__(self, value):
        return value in self.data 
    
    def __iter__(self):
        return iter(self.data)
    
    def __reversed__(self):
        return self.data.reverse()
    
    def count(self):
        return len(self.data)
    
    def index(self, value):
        return self.data.index(value)

In [72]:
another_sample_sequence = ['something', 'else']

In [73]:
print(isinstance(another_sample_sequence, abc.Sequence))

True


Even without registering the object into a virtual subclass, it can sometimes help to inherit from one of the classes provided by `collections` this will ensure that anything you do not implement, will be taken care of by the parent class.  

Another possible motivation of doing isinstance() on abc objects is because, when it comes to passing in data, we typically do not care if the data is passed in as a tuple, or as a list even though each of those data structures has their respective quirks. We just want to iterate over an object and do something to each item. In that regard, whether it's a tuple or a list, what we want is really a Sequence.

### _ChainMaps_

A useful data structure for when you wish to do sequential lookups
<div style='text-align: right; font-size: small;'>
<a href="#data-structures">to top</a>
</div>

In [110]:
from collections import ChainMap, defaultdict

d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4, 'e':5}
d3 = {'a': 6, 'd': 2}
def dflt_fac():
    print('<running factory...>')
    return 'default'
d4 = defaultdict(dflt_fac)

chain = ChainMap(d1, d2, d3, d4)

header_length = 65
print('A ChainMap instance can be seen as a sort of collection of dicts in which look up can be done across all given dicts')
header_print('If the same key exists, only the first found key is returned', chain['d'], header_length)
header_print("If the key doesn't exist, but a default dict exists in the chain", chain['f'], header_length)
chain['a'] = 'virus'
header_print("If a change is made to a key, only the first key is affected", chain.maps, header_length)
header_print("ChainMaps hold a view to the underlying dictionaries", d1, header_length)

A ChainMap instance can be seen as a sort of collection of dicts in which look up can be done across all given dicts
If the same key exists, only the first found key is returned     : 4
<running factory...>
If the key doesn't exist, but a default dict exists in the chain : default
If a change is made to a key, only the first key is affected     : [{'a': 'virus', 'b': 2}, {'c': 3, 'd': 4, 'e': 5}, {'a': 6, 'd': 2}, defaultdict(<function dflt_fac at 0x000002688FAF0D30>, {'f': 'default'})]
ChainMaps hold a view to the underlying dictionaries             : {'a': 'virus', 'b': 2}


### Sets 
<div style='text-align: right; font-size: small;'>
<a href="#data-structures">to top</a>
</div>

In [112]:
l = ['spam', 'spam', 'eggs', 'spam', 'bacon', 'eggs']
s = set(l)
s

{'bacon', 'eggs', 'spam'}

In [120]:
print('sets are not hashable but frozensets are')
f = frozenset(l)
s.add(set(['other', 'food']))

sets are not hashable but frozensets are


TypeError: unhashable type: 'set'

In [121]:
s.add(frozenset(['other', 'food']))

In [122]:
s

{'bacon', 'eggs', frozenset({'food', 'other'}), 'spam'}

Sets are unordered by default so if you wish to get an ordered list of unique values while preserving the order, you can do this instead:

In [123]:
a_l = l.copy()

In [124]:
unique_l_ordered = list(dict.fromkeys(a_l).keys())

In [125]:
unique_l_ordered

['spam', 'eggs', 'bacon']

Since sets are built on hash tables, membership testing is very fast. Therefore, if you need to look for the presence of items within a data structure, please consider using a set to store your data 

In [137]:
from unicodedata import name, digit, numeric, east_asian_width
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}